In [148]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [149]:
class MyDataset(Dataset):
    # 构造函数
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
    # 返回数据集大小
    def __len__(self):
        return self.data_tensor.size(0)
    # 返回索引的数据与标签
    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

In [150]:
from scipy.io import loadmat
file6 = "D:\\320 Deep-learning\\1111_S6_time1000.mat"
data6 = loadmat(file6)
file12 = "D:\\320 Deep-learning\\1111_S12_time1000.mat"
data12 = loadmat(file12)
file24 = "D:\\320 Deep-learning\\1111_S24_time1000.mat"
data24 = loadmat(file24)
""" experiment
X_train = data['Input']
X_train = torch.from_numpy(X_train).float()
y_train = data['Stability']
y_train = y_train.astype(np.float32) / 255
y_train = torch.tensor(y_train)
len_train = len(y_train)
X_test = data['PreInput']
X_test = torch.from_numpy(X_test).float()
y_test = data['PreSta']
y_test = y_test.astype(np.float32) / 255
y_test = torch.tensor(y_test)
len_test = len(y_test) """

#simulation
X6 = data6['Input_Abundance']
X6 = torch.from_numpy(X6).float()
y6 = data6['Std_total']
y6 = -np.log10(y6)
y6 = y6.T
y6 = torch.from_numpy(y6).float()

X12 = data12['Input_RelativeAbundance']
X12 = torch.from_numpy(X12).float()
y12 = data12['Stability']
y12 = torch.from_numpy(y12).float()

X24 = data24['Input_RelativeAbundance']
X24 = torch.from_numpy(X24).float()
y24= data24['Stability']
y24 = torch.from_numpy(y24).float()

t = 500
X_train = X6[0:t,:]
y_train = y6[0:t]

X_test = X6[t:1000,:]
y_test = y6[t:1000]

len_train = len(y_train)
len_test = len(y_test)

# 将数据封装成Dataset
train_dataset = MyDataset(X_train, y_train)

In [151]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 50

""" 需不需要随机采样？ 随机采样和shuffle是不是一样的 """
""" # define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_index)
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, 
                                           sampler = train_sampler, num_workers = num_workers) """

# prepare data loaders
train_dataloader = DataLoader(dataset = train_dataset, # 传入的数据集, 必须参数
                               batch_size = batch_size,       # 输出的batch大小
                               shuffle = True,       # 数据是否打乱
                               num_workers = num_workers)      # 进程数, 0表示只有主进程

#for data,target in train_dataloader:
#    print(target.size(1))

In [152]:
import torch.nn as nn
import torch.nn.functional as F

# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(54, 1)
        self.fc2 = nn.Linear(54, 54)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        #x = F.sigmoid(self.fc1(x))
        return x

# initialize the NN
model = Net()
#print(model)

In [153]:
# specify loss function
criterion = nn.SmoothL1Loss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.Adam(model.parameters(),lr = 0.1)

In [154]:
# number of epochs to train the model
n_epochs = 100
train_loss_min = np.Inf  # set initial "min" to infinity

for epoch in range(n_epochs):
    train_loss = 0
    # train the model #
    loss = 0
    for data, target in train_dataloader: 
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    # calculate average loss over an epoch
    train_loss = train_loss / len_train
    
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    
    # save model
    if train_loss <= train_loss_min:
        print('Train loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        train_loss_min,
        train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss

Epoch: 1 	Training Loss: 7.373371
Train loss decreased (inf --> 7.373371).  Saving model ...
Epoch: 2 	Training Loss: 5.200790
Train loss decreased (7.373371 --> 5.200790).  Saving model ...
Epoch: 3 	Training Loss: 4.534297
Train loss decreased (5.200790 --> 4.534297).  Saving model ...
Epoch: 4 	Training Loss: 4.361402
Train loss decreased (4.534297 --> 4.361402).  Saving model ...
Epoch: 5 	Training Loss: 4.250099
Train loss decreased (4.361402 --> 4.250099).  Saving model ...
Epoch: 6 	Training Loss: 4.146095
Train loss decreased (4.250099 --> 4.146095).  Saving model ...
Epoch: 7 	Training Loss: 4.046939
Train loss decreased (4.146095 --> 4.046939).  Saving model ...
Epoch: 8 	Training Loss: 3.982656
Train loss decreased (4.046939 --> 3.982656).  Saving model ...
Epoch: 9 	Training Loss: 3.929502
Train loss decreased (3.982656 --> 3.929502).  Saving model ...
Epoch: 10 	Training Loss: 3.889799
Train loss decreased (3.929502 --> 3.889799).  Saving model ...
Epoch: 11 	Training Loss

In [155]:

model.load_state_dict(torch.load('model.pt'))
#for name, parameters in model.named_parameters():
#    print(name, ':', parameters.size())
print(model.fc1.weight)

Parameter containing:
tensor([[ 2.3721,  0.6350,  2.0054,  2.9148,  0.1143,  1.1478, -2.7562,  1.1001,
          3.0882, -2.6150,  2.9371,  2.6371, -1.0636, -0.0946, -0.5080, -0.8238,
         -0.6493,  3.9894,  2.6490, -1.5694,  0.9870,  1.7820,  0.9278,  0.0675,
         -2.9977, -5.3900, -1.6041,  4.2010, -2.2948, -1.9697, -6.8755,  0.0745,
          0.5881,  0.9349, -3.2190,  1.1640, -0.7082,  2.0214, -1.2555,  0.8160,
          1.0351,  3.1720, -1.3668, -0.9024,  2.7254, -1.4308,  1.3055,  0.5564,
         -5.4227, -4.2962, -0.6810,  2.5872,  3.7984,  2.5706]],
       requires_grad=True)


In [156]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
correct = 0
OUTPUT = []
S_pre = torch.Tensor(len_test,1)
S_true = torch.Tensor(len_test,1)

for i in range(len_test):
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(X_test[i])
    o = output.tolist()
    OUTPUT.append(o)
    # calculate the loss
    loss = criterion(output, y_test[i])
    # update test loss 
    test_loss += loss.item()
    # use the output to predict y
    if(output>2): S_pre[i] = 0
    else: S_pre[i] = 1
    if(y_test[i]>2): S_true[i] = 0
    else: S_true[i] = 1   
    correct += S_pre[i].eq(S_true[i])

# calculate and print avg test loss
test_loss = test_loss/len_test
correct_ratio = correct/len_test
print('Test Loss: {:.6f}\n'.format(test_loss))
print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
    100*correct_ratio,correct,len_test))


Test Loss: 5.000474


Test Accuracy: 84% (424/500)
